In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import proplot as plot

current_path = !pwd
parent_path = os.path.dirname(current_path[0])
if parent_path not in sys.path: sys.path.append(parent_path)

import utils.model_handling as model_funcs
import utils.dataset_generation as iso_data
import utils.histogram_analysis as hist_funcs
import utils.principal_curvature as curv_funcs
import utils.plotting as plot_funcs

In [ ]:
def gabor_fn(theta, sigma=2, Lambda=10, psi=np.pi/2, gamma=.8, center=(0, 0), size=(28, 28), normalize=True):
    """Returns a gabor filter.
    Args:
        theta (float): Orientation of the sinusoid (in radians).
        sigma (float): std deviation of the Gaussian.
        Lambda (float): Sinusoid wavelengh (1/frequency).
        psi (float): Phase of the sinusoid.
        gamma (float): The ratio between sigma in x-dim over sigma in y-dim (acts
            like an aspect ratio of the Gaussian).
        center (tuple of integers): The position of the filter.
        size (tuple of integers): Image height and width.
        normalize (bool): Whether to normalize the entries. This is computed by
            dividing each entry by the root sum squared of the whole image.
    Returns:
        2D Numpy array: A gabor filter.
    """
    sigma_x = sigma
    sigma_y = sigma / gamma
    xmax, ymax = size
    xmax, ymax = (xmax - 1)/2, (ymax - 1)/2
    xmin = -xmax
    ymin = -ymax
    (y, x) = np.meshgrid(np.arange(ymin, ymax+1), np.arange(xmin, xmax+1))
    # shift the positon
    y -= center[0]
    x -= center[1]
    # Rotation
    x_theta = x * np.cos(theta) + y * np.sin(theta)
    y_theta = -x * np.sin(theta) + y * np.cos(theta)
    # Gabor equation
    gb = np.exp(-.5 * (x_theta ** 2 / sigma_x ** 2 + y_theta ** 2 / sigma_y ** 2)) * np.cos(2 * np.pi / Lambda * x_theta + psi)
    if normalize:
        gb /= np.sqrt(np.sum(gb ** 2)) # root sum squared
        gb -= gb.mean() # make sure the sum is equal to zero
    return gb


def get_network_response_linear(img, weights):
    """Linear network, weights shape should be [img.size, num_neurons]"""
    responses = np.dot(img.reshape(-1)[None, :], weights).T # column vector [num_neurons, 1]
    return responses


def get_unit_response_linear(img, weights, unit_id=None):
    if unit_id is None: # weights are for an individual unit
        response = np.dot(img.reshape(-1)[None, :], weights.reshape(-1)[:, None]) 
    else: # weights are for a network, need to select the individual unit
        response = get_network_response_linear(img, weights)[unit_id]
    return response


def get_unit_response_complex(img, w1, w2):
    u1 = get_unit_response_linear(img, w1)
    u2 = get_unit_response_linear(img, w2)
    response = np.sqrt(u1**2 + u2**2)
    return response


def get_lateral_weights(network_weights):
    return np.matmul(network_weights.T, network_weights) - np.eye(network_weights.shape[1])


def get_unit_response_gdn(img, network_weights, unit_id):
    network_responses = get_network_response_linear(img, network_weights)
    lateral_weights = np.square(get_lateral_weights(network_weights))
    bias = np.mean(lateral_weights)
    weighted_responses = np.matmul(lateral_weights, np.square(network_responses))
    normalization_coefficient = np.sqrt(np.square(bias) + weighted_responses)
    normalized_responses = network_responses / normalization_coefficient
    return normalized_responses[unit_id], lateral_weights


def get_unit_response_softmax(img, network_weights, unit_id):
    network_responses = get_network_response_linear(img, network_weights)
    normalized_responses = np.exp(network_responses) / np.sum(np.exp(network_responses))
    return normalized_responses[unit_id]


def get_unit_response_sparse_coding(img, network_weights, unit_id, num_steps=25, threshold=0.2, step_size=0.05):
    threshold_units = lambda x: np.where(np.greater(x, threshold), x - threshold, np.zeros_like(x))
    forward_drive = get_network_response_linear(img, network_weights)
    lateral_weights = get_lateral_weights(network_weights)
    internal_states = np.zeros((network_weights.shape[1], 1))
    for step_index in range(num_steps):
        inhibitory_input = np.matmul(lateral_weights, threshold_units(internal_states))
        du = forward_drive - inhibitory_input - internal_states
        internal_states = internal_states + step_size * du
    return threshold_units(internal_states)[unit_id]

In [ ]:
experiment_params = dict()
experiment_params['x_range'] = (-2.0, 2.0)
experiment_params['y_range'] = (-2.0, 2.0)
experiment_params['num_images'] = int(40**2)
experiment_params['num_top_neurons'] = 10
experiment_params['num_comparisons_per_neuron'] = 150
experiment_params['target'] = 0.7 * (1 / experiment_params['x_range'][1]) # 70% MEI contrast
experiment_params['target_is_act'] = False
experiment_params['bounds'] = None
experiment_params['measure_between_vecs'] = False#True

experiment_params['num_gauss_fits'] = 20
experiment_params['gauss_thresh'] = 0.2
experiment_params['gauss_confidence_cutoff'] = 0.90 #This percentage of probability density will be preserved

yx_range = (experiment_params['y_range'], experiment_params['x_range'])
num_images_per_edge = int(np.sqrt(experiment_params['num_images']))

experiment_params['target_unit_ids'] = [0]
experiment_params['data_shape'] = [1, 28, 28]
experiment_params['image_scale'] = 1.0

In [ ]:
size = tuple(experiment_params['data_shape'][1:])
normalize = False

theta = np.pi/2; sigma = 3.5; Lambda = size[0]/4; gamma = 0.5; center = [0, 0]

psi = np.pi
gabor1 = gabor_fn(theta, sigma, Lambda, psi, gamma, center, size, normalize)
gabor1 = gabor1 / np.linalg.norm(gabor1)

psi = np.pi/2
gabor2 = gabor_fn(theta, sigma, Lambda, psi, gamma, center, size, normalize)
gabor2 = gabor2 / np.linalg.norm(gabor2)

fig, axs = plot.subplots(nrows=1, ncols=2)
axs[0].imshow(gabor1, cmap='greys_r')
axs[1].imshow(gabor2, cmap='greys_r')
plot.show()

print(f'inner product between gabors = {np.dot(gabor1.reshape(-1)[None, :], gabor2.reshape(-1)[:, None])}')

In [ ]:
num_target_planes = 1
num_comp_planes = 1
plane_abscissae = [gabor1.reshape(-1)] # horizontal axes for the planes
plane_ordinates = [[gabor2.reshape(-1)]] # vertical axes for the planes

contour_dataset = iso_data.get_contour_dataset(
    plane_abscissae,
    plane_ordinates,
    yx_range=yx_range,
    num_images=experiment_params['num_images'],
    image_scale=experiment_params['image_scale'],
    data_shape=experiment_params['data_shape'],
    return_datapoints=True
)

In [ ]:
fig, axs = plot.subplots(nrows=1, ncols=2)
new_shape = (num_images_per_edge, num_images_per_edge)
axs[0].imshow(contour_dataset['proj_datapoints'][:,0].reshape(new_shape))
mappable = axs[1].imshow(contour_dataset['proj_datapoints'][:,1].reshape(new_shape))
axs[1].colorbar(mappable)
axs.format(suptitle='2D Meshgrid')
plot.show()

In [ ]:
datapoints = contour_dataset.pop('all_datapoints')[0][0]
out_shape = (1, 1, 1, num_images_per_edge, num_images_per_edge)
response_images = np.zeros(out_shape)
act = np.zeros(experiment_params['num_images'])
for data_idx in range(datapoints.shape[0]):
    act[data_idx] = get_unit_response_complex(datapoints[data_idx, ...], gabor1, gabor2)
response_images[0, 0, 0, ...] = act.reshape(num_images_per_edge, num_images_per_edge)

num_y, num_x = response_images.shape[-2:]
yx_pts = (contour_dataset['y_pts'].copy(), contour_dataset['x_pts'].copy())
yx_scale_factors, yx_lims = hist_funcs.get_scale_factors(yx_pts, num_y, num_x, experiment_params['bounds'])
preproc_resp_images = hist_funcs.crop_and_mask_activity_map(response_images, yx_lims, contour_dataset, mask=experiment_params['measure_between_vecs'])[0, ...]
iso_curvatures, iso_fits, iso_contours = hist_funcs.iso_response_curvature_poly_fits(
    preproc_resp_images,
    target=experiment_params['target'],
    target_is_act=experiment_params['target_is_act'],
    yx_scale=yx_scale_factors
)

In [ ]:
target_plane_id = 0
comp_plane_id = 0

num_y, num_x = response_images.shape[3:]
yx_pts = (contour_dataset['y_pts'].copy(), contour_dataset['x_pts'].copy())

fig, ax = plot.subplots(nrows=1, ncols=1, sharey=False, sharex=False)

proj_vects = (
    contour_dataset['proj_target_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_comparison_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_orth_vect'][target_plane_id][comp_plane_id],
)
contsf = plot_funcs.plot_contours(
    ax,
    activity=response_images[0, target_plane_id, comp_plane_id, ...],
    yx_pts=yx_pts,
    yx_range=yx_range,
    proj_vects=proj_vects,
    num_levels=10,
    contours=iso_contours[target_plane_id][comp_plane_id],
    fits=iso_fits[target_plane_id][comp_plane_id],
    title=''#f'C={np.round(curvatures[target_plane_id][comp_plane_id], 2)}'
)
proj_matrix = contour_dataset['proj_matrix'][target_plane_id][comp_plane_id]
stim_images = iso_data.inject_data(
    proj_matrix,
    contour_dataset['proj_datapoints'],
    experiment_params['image_scale'],
    experiment_params['data_shape']
)
stim_images = stim_images.reshape(num_images_per_edge, num_images_per_edge, *stim_images.shape[1:])

images_vmin = datapoints.min()
images_vmax = datapoints.max()
overlay_images = []

axis_y_pos = 0
axis_x_pos = 2
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 2
axis_x_pos = 2
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 1
axis_x_pos = 2
offset = [-35, -18]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 2
axis_x_pos = 0
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 0
axis_x_pos = -2
offset = [20, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

ax.format(
    title='Iso-response contours\nComplex cell',
    ylim=experiment_params['y_range'],
    xlim=experiment_params['x_range'],
    xticklabels=[],
    yticklabels=[]
)

plot.show()

In [ ]:
noise_sample = np.random.randn(gabor1.size)
plane_ordinates = [[noise_sample / np.linalg.norm(noise_sample)]] 
contour_dataset = iso_data.get_contour_dataset(
    plane_abscissae,
    plane_ordinates,
    yx_range=yx_range,
    num_images=experiment_params['num_images'],
    image_scale=experiment_params['image_scale'],
    data_shape=experiment_params['data_shape'],
    return_datapoints=True
)

datapoints = contour_dataset.pop('all_datapoints')[0][0]
out_shape = (1, 1, 1, num_images_per_edge, num_images_per_edge)
response_images = np.zeros(out_shape)
act = np.zeros(experiment_params['num_images'])
for data_idx in range(datapoints.shape[0]):
    act[data_idx] = get_unit_response_linear(datapoints[data_idx, ...], gabor1)
response_images[0, 0, 0, ...] = act.reshape(num_images_per_edge, num_images_per_edge)

num_y, num_x = response_images.shape[-2:]
yx_pts = (contour_dataset['y_pts'].copy(), contour_dataset['x_pts'].copy())
yx_scale_factors, yx_lims = hist_funcs.get_scale_factors(yx_pts, num_y, num_x, experiment_params['bounds'])
preproc_resp_images = hist_funcs.crop_and_mask_activity_map(response_images, yx_lims, contour_dataset, mask=experiment_params['measure_between_vecs'])[0, ...]
iso_curvatures, iso_fits, iso_contours = hist_funcs.iso_response_curvature_poly_fits(
    preproc_resp_images,
    target=experiment_params['target'],
    target_is_act=experiment_params['target_is_act'],
    yx_scale=yx_scale_factors
)

In [ ]:
target_plane_id = 0
comp_plane_id = 0

fig, ax = plot.subplots(nrows=1, ncols=1, sharey=False, sharex=False)

proj_vects = (
    contour_dataset['proj_target_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_comparison_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_orth_vect'][target_plane_id][comp_plane_id],
)

contsf = plot_funcs.plot_contours(
    ax,
    activity=response_images[0, target_plane_id, comp_plane_id, ...],
    yx_pts=yx_pts,
    yx_range=yx_range,
    proj_vects=proj_vects,
    num_levels=10,
    contours=iso_contours[target_plane_id][comp_plane_id],
    fits=iso_fits[target_plane_id][comp_plane_id],
    title=''#f'C={np.round(curvatures[target_plane_id][comp_plane_id], 2)}'
)
proj_matrix = contour_dataset['proj_matrix'][target_plane_id][comp_plane_id]
stim_images = iso_data.inject_data(
    proj_matrix,
    contour_dataset['proj_datapoints'],
    experiment_params['image_scale'],
    experiment_params['data_shape']
)
stim_images = stim_images.reshape(num_images_per_edge, num_images_per_edge, *stim_images.shape[1:])

images_vmin = datapoints.min()
images_vmax = datapoints.max()
overlay_images = []

axis_y_pos = 0
axis_x_pos = 2
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 2
axis_x_pos = 2
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 1
axis_x_pos = 2
offset = [-35, -18]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 2
axis_x_pos = 0
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 0
axis_x_pos = -2
offset = [20, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

ax.format(
    title='Iso-response contours\nLinear cell',
    ylim=experiment_params['y_range'],
    xlim=experiment_params['x_range'],
    xticklabels=[],
    yticklabels=[]
)

plot.show()

In [ ]:
n_orientation = 5
n_freq = 5
n_center = 10
n_stds = 2

orientations = np.linspace(0, np.pi, n_orientation)
frequencies = np.linspace(size[0]/4, size[0]/2, n_freq) 
locations_y = np.linspace(-size[0]/5, size[0]/5, int(np.sqrt(n_center)))
locations_x = np.linspace(-size[1]/5, size[1]/5, int(np.sqrt(n_center)))
locations = [[loc_y, loc_x] for loc_y in locations_y for loc_x in locations_x]
gauss_stds = np.linspace(3.5, 4.0, n_stds)
gauss_aspects = [0.5]
phases = [np.pi]
size = tuple(experiment_params['data_shape'][1:])
normalize = False

gabor_bank = []
for freq in frequencies:
    for orientation in orientations:
        for center in locations:
            for aspect in gauss_aspects:
                for std in gauss_stds:
                    for phase in phases:
                        gabor = gabor_fn(orientation, std, freq, phase, aspect, center, size, normalize)
                        gabor = gabor / np.linalg.norm(gabor)
                        gabor_bank.append(gabor.reshape(-1))
gabor_bank = np.stack(gabor_bank, axis=1) # [num_inputs, num_outputs]

In [ ]:
vmin = np.min(gabor_bank)
vmax = np.max(gabor_bank)
num_plots = np.minimum(150, gabor_bank.shape[0])
num_plots_y = int(np.ceil(np.sqrt(num_plots)))
num_plots_x = int(np.floor(np.sqrt(num_plots)))
fig, axs = plot.subplots(nrows=num_plots_y, ncols=num_plots_x, space=0.05, width=4, height=4)
filter_total = 0
for plot_id in  np.ndindex((num_plots_y, num_plots_x)):
    if filter_total < num_plots:
        ax = plot_funcs.clear_axis(axs[plot_id])
        ax.imshow(np.squeeze(gabor_bank[:, filter_total].reshape(size)), vmin=vmin, vmax=vmax, cmap="greys_r")
        filter_total += 1
    else:
        ax = plot_funcs.clear_axis(axs[plot_id])
axs.format(suptitle = 'Gabor filter bank')
plot.show()

In [ ]:
num_target_planes = 1
num_comp_planes = 1
unit_id, comp_id = (0, 2)
plane_abscissae = [gabor_bank[:, unit_id]]
plane_ordinates = [[gabor_bank[:, comp_id]]]

contour_dataset = iso_data.get_contour_dataset(
    plane_abscissae,
    plane_ordinates,
    yx_range=yx_range,
    num_images=experiment_params['num_images'],
    image_scale=experiment_params['image_scale'],
    data_shape=experiment_params['data_shape'],
    return_datapoints=True
)
datapoints = contour_dataset.pop('all_datapoints')[0][0]
out_shape = (1, 1, 1, num_images_per_edge, num_images_per_edge)
response_images = np.zeros(out_shape)
act = np.zeros(experiment_params['num_images'])
for data_idx in range(datapoints.shape[0]):
    act[data_idx], gamma = get_unit_response_gdn(datapoints[data_idx, ...], gabor_bank, unit_id)
response_images[0, 0, 0, ...] = act.reshape(num_images_per_edge, num_images_per_edge)

num_y, num_x = response_images.shape[-2:]
yx_pts = (contour_dataset['y_pts'].copy(), contour_dataset['x_pts'].copy())
yx_scale_factors, yx_lims = hist_funcs.get_scale_factors(yx_pts, num_y, num_x, experiment_params['bounds'])
preproc_resp_images = hist_funcs.crop_and_mask_activity_map(response_images, yx_lims, contour_dataset, mask=experiment_params['measure_between_vecs'])[0, ...]
iso_curvatures, iso_fits, iso_contours = hist_funcs.iso_response_curvature_poly_fits(
    preproc_resp_images,
    target=experiment_params['target'],
    target_is_act=experiment_params['target_is_act'],
    yx_scale=yx_scale_factors
)

In [ ]:
fig, ax = plot.subplots(nrows=1, ncols=1)
mappable = ax.imshow(gamma, cmap='viridis')
ax.colorbar(mappable)
ax.format(title='Divisive normalization weights')
plot.show()

In [ ]:
target_plane_id = 0
comp_plane_id = 0

num_y, num_x = response_images.shape[3:]
yx_pts = (contour_dataset['y_pts'].copy(), contour_dataset['x_pts'].copy())

fig, ax = plot.subplots(nrows=1, ncols=1, sharey=False, sharex=False)

proj_vects = (
    contour_dataset['proj_target_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_comparison_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_orth_vect'][target_plane_id][comp_plane_id],
)

contsf = plot_funcs.plot_contours(
    ax,
    activity=response_images[0, target_plane_id, comp_plane_id, ...],
    yx_pts=yx_pts,
    yx_range=yx_range,
    proj_vects=proj_vects,
    num_levels=10,
    contours=iso_contours[target_plane_id][comp_plane_id],
    fits=iso_fits[target_plane_id][comp_plane_id],
    title=''#f'C={np.round(curvatures[target_plane_id][comp_plane_id], 2)}'
)
proj_matrix = contour_dataset['proj_matrix'][target_plane_id][comp_plane_id]
stim_images = iso_data.inject_data(
    proj_matrix,
    contour_dataset['proj_datapoints'],
    experiment_params['image_scale'],
    experiment_params['data_shape']
)
stim_images = stim_images.reshape(num_images_per_edge, num_images_per_edge, *stim_images.shape[1:])

overlay_images = []

images_vmin = datapoints.min()
images_vmax = datapoints.max()

axis_y_pos = 0
axis_x_pos = 1
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 1
axis_x_pos = 0
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

ax.format(
    title='Iso-response contours\nDivisive normalization',
    ylim=experiment_params['y_range'],
    xlim=experiment_params['x_range'],
    xticklabels=[],
    yticklabels=[]
)

plot.show()

In [ ]:
num_target_planes = 1
num_comp_planes = 1
unit_id, comp_id = (0, 2)
plane_abscissae = [gabor_bank[:, unit_id]]
plane_ordinates = [[gabor_bank[:, comp_id]]]

contour_dataset = iso_data.get_contour_dataset(
    plane_abscissae,
    plane_ordinates,
    yx_range=yx_range,
    num_images=experiment_params['num_images'],
    image_scale=experiment_params['image_scale'],
    data_shape=experiment_params['data_shape'],
    return_datapoints=True
)
datapoints = contour_dataset.pop('all_datapoints')[0][0]
out_shape = (1, 1, 1, num_images_per_edge, num_images_per_edge)
response_images = np.zeros(out_shape)
act = np.zeros(experiment_params['num_images'])
for data_idx in range(datapoints.shape[0]):
    act[data_idx] = get_unit_response_softmax(datapoints[data_idx, ...], gabor_bank, unit_id)
response_images[0, 0, 0, ...] = act.reshape(num_images_per_edge, num_images_per_edge)

num_y, num_x = response_images.shape[-2:]
yx_pts = (contour_dataset['y_pts'].copy(), contour_dataset['x_pts'].copy())
yx_scale_factors, yx_lims = hist_funcs.get_scale_factors(yx_pts, num_y, num_x, experiment_params['bounds'])
preproc_resp_images = hist_funcs.crop_and_mask_activity_map(response_images, yx_lims, contour_dataset, mask=experiment_params['measure_between_vecs'])[0, ...]
iso_curvatures, iso_fits, iso_contours = hist_funcs.iso_response_curvature_poly_fits(
    preproc_resp_images,
    target=experiment_params['target'],
    target_is_act=experiment_params['target_is_act'],
    yx_scale=yx_scale_factors
)

In [ ]:
target_plane_id = 0
comp_plane_id = 0
num_y, num_x = response_images.shape[3:]
yx_pts = (contour_dataset['y_pts'].copy(), contour_dataset['x_pts'].copy())
fig, ax = plot.subplots(nrows=1, ncols=1, sharey=False, sharex=False)
proj_vects = (
    contour_dataset['proj_target_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_comparison_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_orth_vect'][target_plane_id][comp_plane_id],
)
contsf = plot_funcs.plot_contours(
    ax,
    activity=response_images[0, target_plane_id, comp_plane_id, ...],
    yx_pts=yx_pts,
    yx_range=yx_range,
    proj_vects=proj_vects,
    num_levels=10,
    contours=iso_contours[target_plane_id][comp_plane_id],
    fits=iso_fits[target_plane_id][comp_plane_id],
    title=''#f'C={np.round(curvatures[target_plane_id][comp_plane_id], 2)}'
)
proj_matrix = contour_dataset['proj_matrix'][target_plane_id][comp_plane_id]
stim_images = iso_data.inject_data(
    proj_matrix,
    contour_dataset['proj_datapoints'],
    experiment_params['image_scale'],
    experiment_params['data_shape']
)
stim_images = stim_images.reshape(num_images_per_edge, num_images_per_edge, *stim_images.shape[1:])
overlay_images = []
images_vmin = datapoints.min()
images_vmax = datapoints.max()

axis_y_pos = 0
axis_x_pos = 1
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 1
axis_x_pos = 0
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

ax.format(
    title='Iso-response contours\nSoftmax',
    ylim=experiment_params['y_range'],
    xlim=experiment_params['x_range'],
    xticklabels=[],
    yticklabels=[]
)
plot.show()


In [ ]:
num_target_planes = 1
num_comp_planes = 1
unit_id, comp_id = (0, 2)
plane_abscissae = [gabor_bank[:, unit_id]]
plane_ordinates = [[gabor_bank[:, comp_id]]]

contour_dataset = iso_data.get_contour_dataset(
    plane_abscissae,
    plane_ordinates,
    yx_range=yx_range,
    num_images=experiment_params['num_images'],
    image_scale=experiment_params['image_scale'],
    data_shape=experiment_params['data_shape'],
    return_datapoints=True
)
datapoints = contour_dataset.pop('all_datapoints')[0][0]
out_shape = (1, 1, 1, num_images_per_edge, num_images_per_edge)
response_images = np.zeros(out_shape)
act = np.zeros(experiment_params['num_images'])
for data_idx in range(datapoints.shape[0]):
    act[data_idx] = get_unit_response_sparse_coding(datapoints[data_idx, ...], gabor_bank, unit_id)
response_images[0, 0, 0, ...] = act.reshape(num_images_per_edge, num_images_per_edge)

num_y, num_x = response_images.shape[-2:]
yx_pts = (contour_dataset['y_pts'].copy(), contour_dataset['x_pts'].copy())
yx_scale_factors, yx_lims = hist_funcs.get_scale_factors(yx_pts, num_y, num_x, experiment_params['bounds'])
preproc_resp_images = hist_funcs.crop_and_mask_activity_map(response_images, yx_lims, contour_dataset, mask=experiment_params['measure_between_vecs'])[0, ...]
iso_curvatures, iso_fits, iso_contours = hist_funcs.iso_response_curvature_poly_fits(
    preproc_resp_images,
    target=experiment_params['target'],
    target_is_act=experiment_params['target_is_act'],
    yx_scale=yx_scale_factors
)

In [ ]:
target_plane_id = 0
comp_plane_id = 0
num_y, num_x = response_images.shape[3:]
yx_pts = (contour_dataset['y_pts'].copy(), contour_dataset['x_pts'].copy())
fig, ax = plot.subplots(nrows=1, ncols=1, sharey=False, sharex=False)
proj_vects = (
    contour_dataset['proj_target_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_comparison_vect'][target_plane_id][comp_plane_id],
    contour_dataset['proj_orth_vect'][target_plane_id][comp_plane_id],
)
contsf = plot_funcs.plot_contours(
    ax,
    activity=response_images[0, target_plane_id, comp_plane_id, ...],
    yx_pts=yx_pts,
    yx_range=yx_range,
    proj_vects=proj_vects,
    num_levels=10,
    contours=iso_contours[target_plane_id][comp_plane_id],
    fits=iso_fits[target_plane_id][comp_plane_id],
    title=''#f'C={np.round(curvatures[target_plane_id][comp_plane_id], 2)}'
)
proj_matrix = contour_dataset['proj_matrix'][target_plane_id][comp_plane_id]
stim_images = iso_data.inject_data(
    proj_matrix,
    contour_dataset['proj_datapoints'],
    experiment_params['image_scale'],
    experiment_params['data_shape']
)
stim_images = stim_images.reshape(num_images_per_edge, num_images_per_edge, *stim_images.shape[1:])
overlay_images = []
images_vmin = datapoints.min()
images_vmax = datapoints.max()

axis_y_pos = 0
axis_x_pos = 1
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

axis_y_pos = 1
axis_x_pos = 0
offset = [-25, -25]
img = plot_funcs.overlay_image(ax, stim_images, axis_y_pos, axis_x_pos, yx_range, offset, images_vmin, images_vmax)
overlay_images.append(img)

ax.format(
    title='Iso-response contours\nSparse Coding',
    ylim=experiment_params['y_range'],
    xlim=experiment_params['x_range'],
    xticklabels=[],
    yticklabels=[]
)
plot.show()